# **Step 1: Importing the required libraries**

In [ ]:
import numpy as np
import cv2
import os
import glob
import matplotlib.pyplot as plt
from skimage.io import imsave
from google.colab.patches import cv2_imshow
from PIL import Image, ImageOps
from matplotlib import cm
from PIL import ImageFilter 
import matplotlib.image as mpimg

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Step 2: Performing Fuzzy C-means Segmentation technique on Enhanced Covid-19 Chest X-ray images**


In [ ]:
def init_mem_mat(nb_pixels, nb_clusters):
    mem_mat = np.zeros((nb_pixels, nb_clusters))
    x = np.arange(nb_pixels)
    for j in range(nb_clusters):
        xj = x % nb_clusters == j
        mem_mat[xj, j] = 1

    return mem_mat

In [ ]:
def compute_centers(img_mat, mem_mat, fuzzy):
    num = np.dot(img_mat, mem_mat ** fuzzy)
    dem = np.sum(mem_mat ** fuzzy, axis=0)

    return num / dem

In [ ]:
def update_mem_mat(ctr_mat, img_mat, fuzzy):
    ctr_mat_mesh, img_mat_mesh = np.meshgrid(ctr_mat, img_mat)
    power = 2. / (fuzzy - 1)
    p1 = abs(img_mat_mesh - ctr_mat_mesh) ** power
    p2 = np.sum((1. / abs(img_mat_mesh - ctr_mat_mesh)) ** power, axis=1)

    return 1. / (p1 * p2[:, None])

In [ ]:
folder='/content/drive/MyDrive/Plant_disease_dataset/Apple/Apple___healthy'
def load_images_from_folder(folder):
  images = []
count=0
for f in os.listdir(folder):
  img = cv2.imread(os.path.join(folder,f))
  #cv2_imshow(img)
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  #plt.show()
  #cv2_imshow(gray)
  K = 2
  N = gray.size
  m = 2
  eps = 0.3
  max_i = 100
  X = gray.flatten().astype('float')
  U = init_mem_mat(N, K)
  i = 0
  while True:
    C = compute_centers(X, U, m)

    # Save initial membership matrix
    old_U = np.copy(U)

    # Update coefficients for each pixel
    U = update_mem_mat(C, X, m)

    # Difference between initial mem matrix and new one
    d = np.sum(abs(U - old_U))
    #print(str(i) + " - d = " + str(d))

    # Check convergence
    if d < eps or i > max_i:
        break
    i += 1
  count += 1
  #cv2_imshow(gray)
  seg = np.argmax(U, axis=1)
  seg = seg.reshape(gray.shape).astype('int')
  #print(type(seg))
  imsave('/content/drive/MyDrive/Segmented_images/Fuzzy C- means/Apple_healthy_Segmented/Fuzzy-C_output'+str(count)+'.JPG',seg)
  #cv2_imshow(seg)
  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: UserWarning: /content/drive/MyDrive/Segmented_images/Fuzzy C- means/Apple_healthy_Segmented/Fuzzy-C_output1.JPG is a low contrast image
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: UserWarning: /content/drive/MyDrive/Segmented_images/Fuzzy C- means/Apple_healthy_Segmented/Fuzzy-C_output2.JPG is a low contrast image
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: UserWarning: /content/drive/MyDrive/Segmented_images/Fuzzy C- means/Apple_healthy_Segmented/Fuzzy-C_output3.JPG is a low contrast image
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: UserWarning: /content/drive/MyDrive/Segmented_images/Fuzzy C- means/Apple_healthy_Segmented/Fuzzy-C_output4.JPG is a low contrast image
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: UserWarning: /content/drive/MyDrive/Segmented_images/Fuzzy C- means/Apple_healthy_Segmented/Fuzzy-C_output5.JPG is a low contrast i

In [ ]:
if __name__=="__main__":
    load_images_from_folder(folder)